In [4]:
m4.data.read_df().tail(5)

,date,instrument,ATR
1745,2015-02-27 14:40:00,000001.SZA,0.030817
1746,2015-02-27 14:45:00,000001.SZA,0.030759
1747,2015-02-27 14:50:00,000001.SZA,0.030705
1748,2015-02-27 14:55:00,000001.SZA,0.029940
1749,2015-02-27 15:00:00,000001.SZA,0.027802


In [1]:
# 本代码由可视化策略环境自动生成 2021年8月22日14:41
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m3_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    df = input_1.read_df().set_index('date')
    
    def resample(df,period):
        # https://pandas-docs.github.io/pandas-docs-travis/timeseries.html#offset-aliases
        Xmin_df=pd.DataFrame()
        Xmin_df['open'] = df['open'].resample(period, how='first')
        Xmin_df['high'] = df['high'].resample(period, how='max')
        Xmin_df['low'] = df['low'].resample(period, how='min')
        Xmin_df['close'] = df['close'].resample(period, how='last')
        Xmin_df['volume'] = df['volume'].resample(period, how='sum')
        Xmin_df['amount'] = df['amount'].resample(period, how='sum')
        Xmin_df.dropna(inplace=True)
        return Xmin_df
    
    resample_df = df.groupby('instrument').apply(lambda x:resample(x, '5min')).reset_index()
    
    data_1 = DataSource.write_df(resample_df)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m3_post_run_bigquant_run(outputs):
    return outputs

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m21_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    input_df = input_1.read_df()
    import talib

    def cal_kama(df):
        # 取出close_0列的数据转化为float
        high = np.array([float(x) for x in df['high']])
        low = np.array([float(x) for x in df['low']])
        close = np.array([float(x) for x in df['close']])
         # 调用talib计算MACD指标
        df['ATR'] = talib.ATR(high, low, close, timeperiod=14)
        return df[['date','instrument','ATR']]
    
    result = input_df.groupby('instrument').apply(cal_kama)
    
    data_1 = DataSource.write_df(result)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m21_post_run_bigquant_run(outputs):
    return outputs


m1 = M.instruments.v2(
    start_date='2015-01-01 09:30:00',
    end_date='2015-03-01 15:00:00',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m20 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
close
open
high
low
volume
amount"""
)

m2 = M.use_datasource.v1(
    instruments=m1.data,
    features=m20.data,
    datasource_id='bar1m_000001.SZA',
    start_date='',
    end_date=''
)

m3 = M.cached.v3(
    input_1=m2.data,
    run=m3_run_bigquant_run,
    post_run=m3_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m21 = M.cached.v3(
    input_1=m3.data_1,
    run=m21_run_bigquant_run,
    post_run=m21_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m4 = M.dropnan.v2(
    input_data=m21.data_1
)


[2020-11-02 17:35:47.693662] INFO: moduleinvoker: instruments.v2 开始运行..

[2020-11-02 17:35:47.787964] INFO: moduleinvoker: instruments.v2 运行完成[0.094301s].

[2020-11-02 17:35:47.790500] INFO: moduleinvoker: input_features.v1 开始运行..

[2020-11-02 17:35:49.194256] INFO: moduleinvoker: input_features.v1 运行完成[1.403735s].

[2020-11-02 17:35:49.203052] INFO: moduleinvoker: use_datasource.v1 开始运行..

...

[2020-11-02 17:35:50.390907] INFO: moduleinvoker: cached.v3 运行完成[0.136338s].

[2020-11-02 17:35:50.427776] INFO: moduleinvoker: dropnan.v2 开始运行..

[2020-11-02 17:35:50.477932] INFO: dropnan: /data, 1736/1750

[2020-11-02 17:35:50.538110] INFO: dropnan: 行数: 1736/1750

[2020-11-02 17:35:50.542193] INFO: moduleinvoker: dropnan.v2 运行完成[0.114411s].